## Overview

This notebook is based on a workshop tutorial originally given at FOCAPD in 2019.
It demonstrated the following capabilities:

- Construct a steady-state flowsheet using the IDAES unit model library
- Connecting unit models in a  flowsheet using Arcs
- Using the SequentialDecomposition tool to initialize a flowsheet with recycle
- Fomulate and solve an optimization problem
    - Defining an objective function
    - Setting variable bounds
    - Adding additional constraints 

Changes were made to use the notebook to demonstrate use of the IDAES Data Management Framework (DMF):
- The underlying Python modules were adapted to use the DMF for loading property data
- The optimization problem was removed to shorten the example

## Problem Statement

Hydrodealkylation is a chemical reaction that often involves reacting
an aromatic hydrocarbon in the presence of hydrogen gas to form a
simpler aromatic hydrocarbon devoid of functional groups,. In this
example, toluene will be reacted with hydrogen gas at high temperatures
 to form benzene via the following reaction:

**C<sub>6</sub>H<sub>5</sub>CH<sub>3</sub> + H<sub>2</sub> → C<sub>6</sub>H<sub>6</sub> + CH<sub>4</sub>**


This reaction is often accompanied by an equilibrium side reaction
which forms diphenyl, which we will neglect for this example.

This example is based on the 1967 AIChE Student Contest problem as
present by Douglas, J.M., Chemical  Design of Chemical Processes, 1988,
McGraw-Hill.

The flowsheet that we will be using for this module is shown below with the stream conditions. We will be processing toluene and hydrogen to produce at least 370 TPY of benzene. As shown in the flowsheet, there are two flash tanks, F101 to separate out the non-condensibles and F102 to further separate the benzene-toluene mixture to improve the benzene purity.  Note that typically a distillation column is required to obtain high purity benzene but that is beyond the scope of this workshop. The non-condensibles separated out in F101 will be partially recycled back to M101 and the rest will be either purged or combusted for power generation.We will assume ideal gas for this flowsheet. The properties required for this module are available in the same directory:

- hda_ideal_VLE.py
- hda_reaction.py

The state variables chosen for the property package are **flows of component by phase, temperature and pressure**. The components considered are: **toluene, hydrogen, benzene and methane**. Therefore, every stream has 8 flow variables, 1 temperature and 1 pressure variable. 

### Data Management Framework

In this version of the example, the DMF "workspace" is used to store these state variables. This involves the use of additional files:

- config.yaml - DMF Configuration
- resourcedb.json - The DMF's file-based datastore, where the values and their provenance are stored.


![](module_2_flowsheet.png)

Importing required pyomo and idaes components
-----------
To construct a flowsheet, we will need several components from the pyomo and idaes package. Let us first import the following components from Pyomo:
- Constraint (to write constraints)
- Var (to declare variables)
- ConcreteModel (to create the concrete model object)
- Expression (to evaluate values as a function of variables defined in the model)
- Objective (to define an objective function for optimization)
- SolverFactory (to solve the problem)
- TransformationFactory (to apply certain transformations)
- Arc (to connect two unit models)
- SequentialDecomposition (to initialize the flowsheet in a sequential mode)

For further details on these components, please refer to the pyomo documentation: https://pyomo.readthedocs.io/en/latest/


In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.network import Arc, SequentialDecomposition

From idaes, we will be needing the FlowsheetBlock and the following unit models:
- Mixer
- Heater
- StoichiometricReactor
- <span style="color:blue">**Flash**</span>
- Separator (splitter) 
- PressureChanger

In [2]:
from idaes.core import FlowsheetBlock

In [3]:
from idaes.generic_models.unit_models import (PressureChanger,
                               Mixer,
                               Separator as Splitter,
                               Heater,
                               StoichiometricReactor)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, import the remaining unit models highlighted in blue above and run the cell using `Shift+Enter` after typing in the code. 
</div>


In [4]:
from idaes.generic_models.unit_models import Flash

We will also be needing some utility tools to put together the flowsheet and calculate the degrees of freedom. 

In [7]:
from idaes.generic_models.unit_models.pressure_changer import ThermodynamicAssumption
from idaes.core.util.model_statistics import degrees_of_freedom

Importing required thermo and reaction package
-----------


The final set of imports are to import the thermo and reaction package for the HDA process. We have created a custom thermo package that assumes Ideal Gas with support for VLE. 

The reaction package here is very simple as we will be using only a StochiometricReactor and the reaction package consists of the stochiometric coefficients for the reaction and the parameter for the heat of reaction. 

Let us import the following modules and they are in the same directory as this jupyter notebook:
      <ul>
         <li>hda_ideal_VLE as thermo_props</li>
         <li>hda_reaction as reaction_props </li>
      </ul>
</div>

In [8]:
import hda_ideal_VLE as thermo_props
import hda_reaction as reaction_props

2020-04-04 16:25:29,103 [WARNING] idaes.dmf.dmfbase: Path to built HTML documentation is not set. The DMF "help" command will not work. To set this path, set "htmldocs" in the DMF configuration file.


Constructing the Flowsheet
----------------------------------

We have now imported all the components, unit models, and property modules we need to construct a flowsheet. Let us create a ConcreteModel and add the flowsheet block as we did in module 1. 

In [9]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

We now need to add the property packages to the flowsheet. Unlike Module 1, where we only had a thermo property package, for this flowsheet we will also need to add a reaction property package. 

In [10]:
m.fs.thermo_params = thermo_props.HDAParameterBlock()
m.fs.reaction_params = reaction_props.HDAReactionParameterBlock(
        default={"property_package": m.fs.thermo_params})

## Viewing imported properties with the DMF

Since the property package above used the DMF to load the constant values used for calculating property values, we can look at the provenance and data programmatically.

In [11]:
from idaes.dmf.resource import TidyUnitData as TDU
prop_name = 'dh_vap'
rsrc = m.fs.thermo_params.get_property_resource(prop_name)
print(f"Property {prop_name} source: {rsrc.formatted_source()}")
print(f"Property {prop_name} data:")
TDU(rsrc.data).table

Property dh_vap source: Reid, R. C., Prausnitz, J. M., & Poling, B. E. (1987). The properties of gases and liquids. ISBN: 9780070517998 Date: 1987
Property dh_vap data:


,compound,value
0,benzene,33870.0
1,toluene,38262.0
2,hydrogen,0.0
3,methane,0.0
4,diphenyl,62710.0


Adding Unit Models
-----

Let us start adding the unit models we have imported to the flowsheet. Here, we are adding the Mixer (assigned a name M101) and a Heater (assigned a name H101). Note that, all unit models need to be given a property package argument. In addition to that, there are several arguments depending on the unit model, please refer to the documentation for more details (https://idaes-pse.readthedocs.io/en/latest/models/index.html). For example, the Mixer unit model here is given a `list` consisting of names to the three inlets. 

In [12]:
m.fs.M101 = Mixer(default={"property_package": m.fs.thermo_params,
                           "inlet_list": ["toluene_feed", "hydrogen_feed", "vapor_recycle"]})

m.fs.H101 = Heater(default={"property_package": m.fs.thermo_params,
                            "has_pressure_change": False,
                            "has_phase_equilibrium": True})

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Let us now add the StoichiometricReactor(assign the name R101) and pass the following arguments:
      <ul>
         <li>"property_package": m.fs.thermo_params</li>
         <li>"reaction_package": m.fs.reaction_params </li>
         <li>"has_heat_of_reaction": True </li>
         <li>"has_heat_transfer": True</li>
         <li>"has_pressure_change": False</li>
      </ul>
</div>

In [13]:
m.fs.R101 = StoichiometricReactor(
            default={"property_package": m.fs.thermo_params,
                     "reaction_package": m.fs.reaction_params,
                     "has_heat_of_reaction": True,
                     "has_heat_transfer": True,
                     "has_pressure_change": False})

Let us now add the Flash(assign the name F101) and pass the following arguments:
      <ul>
         <li>"property_package": m.fs.thermo_params</li>
         <li>"has_heat_transfer": True</li>
         <li>"has_pressure_change": False</li>
      </ul>

In [14]:
m.fs.F101 = Flash(default={"property_package": m.fs.thermo_params,
                               "has_heat_transfer": True,
                               "has_pressure_change": True})

Let us now add the Splitter(S101), PressureChanger(C101) and the second Flash(F102). 

In [15]:
m.fs.S101 = Splitter(default={"property_package": m.fs.thermo_params,
                               "ideal_separation": False,
                               "outlet_list": ["purge", "recycle"]})
    

m.fs.C101 = PressureChanger(default={
            "property_package": m.fs.thermo_params,
            "compressor": True,
            "thermodynamic_assumption": ThermodynamicAssumption.isothermal})
    
m.fs.F102 = Flash(default={"property_package": m.fs.thermo_params,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

Connecting Unit Models using Arcs
-----
We have now added all the unit models we need to the flowsheet. However, we have not yet specifed how the units are to be connected. To do this, we will be using the `Arc` which is a pyomo component that takes in two arguments: `source` and `destination`. Let us connect the outlet of the mixer(M101) to the inlet of the heater(H101). 

In [16]:
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)


![](module_2_flowsheet.png) 

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, connect the H101 outlet to the R101 inlet using the cell above as a guide. 
</div>



In [17]:
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)

We will now be connecting the rest of the flowsheet as shown below. Notice how the outlet names are different for the flash tanks F101 and F102 as they have a vapor and a liquid outlet. 

In [18]:
m.fs.s05 = Arc(source=m.fs.R101.outlet, destination=m.fs.F101.inlet)
m.fs.s06 = Arc(source=m.fs.F101.vap_outlet, destination=m.fs.S101.inlet)
m.fs.s08 = Arc(source=m.fs.S101.recycle, destination=m.fs.C101.inlet)
m.fs.s09 = Arc(source=m.fs.C101.outlet,
               destination=m.fs.M101.vapor_recycle)
m.fs.s10 = Arc(source=m.fs.F101.liq_outlet, destination=m.fs.F102.inlet)

We have now connected the unit model block using the arcs. However, each of these arcs link to ports on the two unit models that are connected. In this case, the ports consist of the state variables that need to be linked between the unit models. Pyomo provides a convenient method to write these equality constraints for us between two ports and this is done as follows:

In [19]:
TransformationFactory("network.expand_arcs").apply_to(m)

Adding expressions to compute purity and operating costs
---
In this section, we will add a few Expressions that allows us to evaluate the performance. Expressions provide a convenient way of calculating certain values that are a function of the variables defined in the model. For more details on Expressions, please refer to: https://pyomo.readthedocs.io/en/latest/pyomo_modeling_components/Expressions.html

For this flowsheet, we are interested in computing the purity of the product Benzene stream (i.e. the mole fraction) and the operating cost which is a sum of the cooling and heating cost. 

Let us first add an Expression to compute the mole fraction of benzene in the `vap_outlet` of F102 which is our product stream. Please note that the var flow_mol_phase_comp has the index - [time, phase, component]. As this is a steady-state flowsheet, the time index by default is 0. The valid phases are ["Liq", "Vap"]. Similarly the valid component list is ["benzene", "toluene", "hydrogen", "methane"].

In [20]:
m.fs.purity = Expression(
        expr=m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"] /
        (m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"]
         + m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "toluene"]))

Now, let us add an expression to compute the cooling cost assuming a cost of 0.212E-4 $/kW. Note that cooling utility is required for the reactor (R101) and the first flash (F101). 

In [21]:
m.fs.cooling_cost = Expression(expr=0.212e-7 * (-m.fs.F101.heat_duty[0]) +
                                   0.212e-7 * (-m.fs.R101.heat_duty[0]))


Now, let us add an expression to compute the heating cost assuming the utility cost as follows:
      <ul>
         <li>2.2E-4 dollars/kW for H101</li>
         <li>1.9E-4 dollars/kW for F102</li>
      </ul>
Note that the heat duty is in units of watt (J/s). 

In [22]:
m.fs.heating_cost = Expression(expr=2.2e-7 * m.fs.H101.heat_duty[0] +
                                   1.9e-7 * m.fs.F102.heat_duty[0])

Let us now add an expression to compute the total operating cost per year which is basically the sum of the cooling and heating cost we defined above. 

In [23]:
m.fs.operating_cost = Expression(expr=(3600 * 24 * 365 *
                                           (m.fs.heating_cost +
                                            m.fs.cooling_cost)))

Fixing feed conditions
---

Let us first check how many degrees of freedom exist for this flowsheet using the `degrees_of_freedom` tool we imported earlier. 

In [24]:
print(degrees_of_freedom(m))

49


We will now be fixing the toluene feed stream to the conditions shown in the flowsheet above. Please note that though this is a pure toluene feed, the remaining components are still assigned a very small non-zero value to help with convergence and initializing. 

In [25]:
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(0.30)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.temperature.fix(303.2)
m.fs.M101.toluene_feed.pressure.fix(350000)


Similarly, let us fix the hydrogen feed to the following conditions in the next cell:
      <ul>
         <li>F<sub>H2</sub> = 0.30 mol/s</li>
         <li>F<sub>CH4</sub> = 0.02 mol/s</li>
         <li>Remaining components = 1e-5 mol/s</li>
         <li>T = 303.2 K</li>
         <li>P = 350000 Pa</li>
      </ul>



In [26]:
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(0.30)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(0.02)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.hydrogen_feed.temperature.fix(303.2)
m.fs.M101.hydrogen_feed.pressure.fix(350000)

Fixing unit model specifications
---

Now that we have fixed our inlet feed conditions, we will now be fixing the operating conditions for the unit models in the flowsheet. Let us set set the H101 outlet temperature to 600 K. 

In [27]:
m.fs.H101.outlet.temperature.fix(600)

For the StoichiometricReactor, we have to define the conversion in terms of toluene. This requires us to create a new variable for specifying the conversion and adding a Constraint that defines the conversion with respect to toluene. The second degree of freedom for the reactor is to define the heat duty. In this case, let us assume the reactor to be adiabatic i.e. Q = 0. 

In [28]:
m.fs.R101.conversion = Var(initialize=0.75, bounds=(0, 1))

m.fs.R101.conv_constraint = Constraint(
    expr=m.fs.R101.conversion*m.fs.R101.inlet.
    flow_mol_phase_comp[0, "Vap", "toluene"] ==
    (m.fs.R101.inlet.flow_mol_phase_comp[0, "Vap", "toluene"] -
     m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "toluene"]))

m.fs.R101.conversion.fix(0.75)
m.fs.R101.heat_duty.fix(0)

The Flash conditions for F101 can be set as follows. 

In [29]:
m.fs.F101.vap_outlet.temperature.fix(325.0)
m.fs.F101.deltaP.fix(0)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Set the conditions for Flash F102 to the following conditions:
      <ul>
         <li>T = 375 K</li>
         <li>deltaP = -200000</li>
      </ul>

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [30]:
m.fs.F102.vap_outlet.temperature.fix(375)
m.fs.F102.deltaP.fix(-200000)

Let us fix the purge split fraction to 20% and the outlet pressure of the compressor is set to 350000 Pa. 

In [31]:
m.fs.S101.split_fraction[0, "purge"].fix(0.2)
m.fs.C101.outlet.pressure.fix(350000)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
We have now defined all the feed conditions and the inputs required for the unit models. The system should now have 0 degrees of freedom i.e. should be a square problem. Please check that the degrees of freedom is 0. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [32]:
print(degrees_of_freedom(m))

20


Initialization
------------------

This section will demonstrate how to use the built-in sequential decomposition tool to initialize our flowsheet.

![](module_2_flowsheet.png) 


Let us first create an object for the SequentialDecomposition and specify our options for this. 

In [33]:
seq = SequentialDecomposition()
seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Wegstein"
seq.options.iterLim = 5

# Using the SD tool
G = seq.create_graph(m)
heuristic_tear_set = seq.tear_set_arcs(G, method="heuristic")
order = seq.calculation_order(G)

Which is the tear stream? Display tear set and order

In [34]:
for o in heuristic_tear_set:
    print(o.name)

fs.s03


What sequence did the SD tool determine to solve this flowsheet with the least number of tears? 

In [35]:
for o in order:
    print(o[0].name)

fs.H101
fs.R101
fs.F101
fs.S101
fs.C101
fs.M101


 

![](module_2_tear_stream.png) 


The SequentialDecomposition tool has determined that the tear stream is the mixer outlet. We will need to provide a reasonable guess for this.

In [36]:
tear_guesses = {
        "flow_mol_phase_comp": {
                (0, "Vap", "benzene"): 1e-5,
                (0, "Vap", "toluene"): 1e-5,
                (0, "Vap", "hydrogen"): 0.30,
                (0, "Vap", "methane"): 0.02,
                (0, "Liq", "benzene"): 1e-5,
                (0, "Liq", "toluene"): 0.30,
                (0, "Liq", "hydrogen"): 1e-5,
                (0, "Liq", "methane"): 1e-5},
        "temperature": {0: 303},
        "pressure": {0: 350000}}

# Pass the tear_guess to the SD tool
seq.set_guesses_for(m.fs.H101.inlet, tear_guesses)

Next, we need to tell the tool how to initialize a particular unit. We will be writing a python function which takes in a "unit" and calls the initialize method on that unit. 

In [37]:
def function(unit):
        unit.initialize(outlvl=1)

We are now ready to initialize our flowsheet in a sequential mode. Note that we specifically set the iteration limit to be 5 as we are trying to use this tool only to get a good set of initial values such that IPOPT can then take over and solve this flowsheet for us. 

In [38]:
seq.run(m, function)

2020-04-04 16:26:21 [INFO] idaes.init.fs.H101.control_volume: Initialization Complete
2020-04-04 16:26:21 [INFO] idaes.init.fs.H101: Initialization Step 1 Complete.
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: Ipopt 3.12.13: tol=1e-06
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: ******************************************************************************
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: This program contains Ipopt, a library for large-scale nonlinear optimization.
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: Ipopt is released as open source code under the Eclipse Public License (EPL).
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: For more information visit http://projects.coin-or.org/Ipopt
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: This version of Ipopt was compiled from source code available at
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: https://github.com/IDAES/Ipopt as part of the Institute for the Design of
2020-04-04 16:26:21 [DEBUG] ida

2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: 34r 0.0000000e+00 2.25e+01 8.30e+06  -3.1 2.72e+01    -  1.00e+00 5.90e-02f  1
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: 35r 0.0000000e+00 2.06e+02 5.10e+08  -3.1 6.80e+00    -  4.91e-01 1.00e+00f  1
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: 36r 0.0000000e+00 2.05e+02 5.08e+08  -3.1 4.33e+00   0.0 3.68e-03 8.68e-03h  1
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: 37r 0.0000000e+00 2.05e+02 5.07e+08  -3.1 8.78e+00   0.4 2.89e-03 1.49e-03h  1
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: 38r 0.0000000e+00 2.04e+02 5.06e+08  -3.1 2.34e+00   0.9 1.69e-03 5.14e-03h  1
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: 39r 0.0000000e+00 1.89e+02 4.69e+08  -3.1 1.96e-04   8.7 3.18e-02 2.51e-01h  1
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: 40r 0.0000000e+00 1.81e+02 4.48e+08  -3.1 5.31e-04   

2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: Overall NLP error.......:   2.6005810093435671e-09    1.6968935270256225e-05
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: Number of objective function evaluations             = 93
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: Number of objective gradient evaluations             = 7
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: Number of equality constraint evaluations            = 93
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: Number of inequality constraint evaluations          = 0
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: Number of equality constraint Jacobian evaluations   = 90
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: Number of inequality constraint Jacobian evaluations = 0
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.H101: Number of Lagrangian Hessian evaluations             = 88
20

2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.R101: 23  0.0000000e+00 1.27e-07 1.66e-08  -2.5 7.04e-03    -  1.00e+00 1.00e+00h  1
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.R101: 24  0.0000000e+00 2.24e-08 3.09e-11  -3.8 3.66e-04    -  1.00e+00 1.00e+00h  1
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.R101: 25  0.0000000e+00 2.24e-08 3.74e-13  -5.7 7.99e-05    -  1.00e+00 1.00e+00H  1
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.R101: 26  0.0000000e+00 7.45e-09 1.75e-14  -7.0 3.17e-04    -  1.00e+00 1.00e+00H  1
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.R101: Number of Iterations....: 26
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.R101: (scaled)                 (unscaled)
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.R101: Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.R101: Dual infeasibility......:   1.7534166437736565e-14    1.7534166437736565e-14
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.R101: Constraint violation....

2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.F101: Number of equality constraint Jacobian evaluations   = 7
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.F101: Number of inequality constraint Jacobian evaluations = 0
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.F101: Number of Lagrangian Hessian evaluations             = 6
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.F101: Total CPU secs in IPOPT (w/o function evaluations)   =      0.002
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.F101: Total CPU secs in NLP function evaluations           =      0.000
2020-04-04 16:26:21 [DEBUG] idaes.solve.fs.F101: EXIT: Optimal Solution Found.
2020-04-04 16:26:21 [INFO] idaes.init.fs.F101: Initialization Step 2 optimal - Optimal Solution Found.
2020-04-04 16:26:21 [INFO] idaes.init.fs.F101: Initialization Complete: optimal - Optimal Solution Found
2020-04-04 16:26:22 [DEBUG] idaes.solve.fs.S101: Ipopt 3.12.13:
2020-04-04 16:26:22 [DEBUG] idaes.solve.fs.S101: ****************************************************

...

2020-04-04 16:26:29 [DEBUG] idaes.solve.fs.R101: Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
2020-04-04 16:26:29 [DEBUG] idaes.solve.fs.R101: Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.
2020-04-04 16:26:29 [DEBUG] idaes.solve.fs.R101: This version of Ipopt was compiled using HSL, a collection of Fortran codes
2020-04-04 16:26:29 [DEBUG] idaes.solve.fs.R101: for large-scale scientific computation.  All technical papers, sales and
2020-04-04 16:26:29 [DEBUG] idaes.solve.fs.R101: publicity material resulting from use of the HSL codes within IPOPT must
2020-04-04 16:26:29 [DEBUG] idaes.solve.fs.R101: contain the following acknowledgement:
2020-04-04 16:26:29 [DEBUG] idaes.solve.fs.R101: HSL, a collection of Fortran codes for large-scale scientific
2020-04-04 16:26:29 [DEBUG] idaes.solve.fs.R101: computation. See http://www.hsl.rl.ac.uk.
2020-04-04 16:26:29 [DEBUG] idaes.solve.fs.R101: *********************************************

2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.F101: HSL, a collection of Fortran codes for large-scale scientific
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.F101: computation. See http://www.hsl.rl.ac.uk.
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.F101: ******************************************************************************
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.F101: This is Ipopt version 3.12.13, running with linear solver ma27.
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.F101: Number of nonzeros in equality constraint Jacobian...:      116
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.F101: Number of nonzeros in inequality constraint Jacobian.:        0
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.F101: Number of nonzeros in Lagrangian Hessian.............:      112
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.F101: Total number of variables............................:       37
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.F101: variables with only lower bounds:        0
2020-04-04 16

2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.S101: variables with only lower bounds:        0
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.S101: variables with lower and upper bounds:       20
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.S101: variables with only upper bounds:        0
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.S101: Total number of equality constraints.................:       13
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.S101: Total number of inequality constraints...............:        0
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.S101: inequality constraints with only lower bounds:        0
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.S101: inequality constraints with lower and upper bounds:        0
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.S101: inequality constraints with only upper bounds:        0
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.S101: iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.

2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.C101: 6  0.0000000e+00 4.58e-09 1.33e-20  -7.0 1.24e-02    -  1.00e+00 1.00e+00h  1
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.C101: Number of Iterations....: 6
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.C101: (scaled)                 (unscaled)
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.C101: Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.C101: Dual infeasibility......:   1.3340768919255230e-20    1.3340768919255230e-20
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.C101: Constraint violation....:   8.3827450231535341e-12    4.5824890548828989e-09
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.C101: Complementarity.........:   9.0935223609295285e-08    9.0935223609295285e-08
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.C101: Overall NLP error.......:   9.0935223609295285e-08    9.0935223609295285e-08
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.C101: Number of objective function eva

2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.M101: Number of equality constraint Jacobian evaluations   = 11
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.M101: Number of inequality constraint Jacobian evaluations = 0
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.M101: Number of Lagrangian Hessian evaluations             = 10
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.M101: Total CPU secs in IPOPT (w/o function evaluations)   =      0.003
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.M101: Total CPU secs in NLP function evaluations           =      0.000
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.M101: EXIT: Optimal Solution Found.
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.M101:
2020-04-04 16:26:30 [INFO] idaes.init.fs.M101: Initialization Complete: optimal - Optimal Solution Found
2020-04-04 16:26:30 [INFO] idaes.init.fs.F102.control_volume: Initialization Complete
2020-04-04 16:26:30 [INFO] idaes.init.fs.F102: Initialization Step 1 Complete.
2020-04-04 16:26:30 [DEBUG] idaes.solve.fs.F102: Ipo

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
We have now initialized the flowsheet. Let us run the flowsheet in a simulation mode to look at the results. To do this, complete the last line of code where we pass the model to the solver. You will need to type the following:
    
results = solver.solve(m, tee=True)

Use Shift+Enter to run the cell once you have typed in your code. 
</div>



In [39]:
# Create the solver object
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6, 'max_iter': 5000}

# Solve the model
results = solver.solve(m, tee=True)

Ipopt 3.12.13: tol=1e-06
max_iter=5000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
       

  82  0.0000000e+00 1.27e+04 1.13e+15  -1.0 2.83e+04    -  9.84e-01 1.97e-03h  9
  83  0.0000000e+00 1.27e+04 1.13e+15  -1.0 2.82e+04    -  1.00e+00 1.97e-03h  9
  84  0.0000000e+00 1.27e+04 1.13e+15  -1.0 2.81e+04    -  9.88e-01 1.97e-03h  9
  85  0.0000000e+00 1.27e+04 1.13e+15  -1.0 2.80e+04    -  1.00e+00 1.97e-03h  9
  86  0.0000000e+00 1.26e+04 1.13e+15  -1.0 2.79e+04    -  9.92e-01 1.97e-03h  9
  87  0.0000000e+00 1.26e+04 1.13e+15  -1.0 2.79e+04    -  1.00e+00 1.97e-03h  9
  88  0.0000000e+00 1.26e+04 1.13e+15  -1.0 2.78e+04    -  9.96e-01 1.97e-03h  9
  89  0.0000000e+00 1.26e+04 1.13e+15  -1.0 2.77e+04    -  1.00e+00 1.97e-03h  9
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  90  0.0000000e+00 1.25e+04 1.13e+15  -1.0 2.76e+04    -  9.99e-01 1.97e-03h  9
  91  0.0000000e+00 2.69e+05 1.16e+15  -1.0 2.76e+04    -  1.00e+00 5.04e-01w  1
  92  0.0000000e+00 1.24e+05 3.14e+15  -1.0 3.67e+04    -  6.55e-01 5.47e-01w  1
  93  0.0000000e+00 3.72e+05

 175  0.0000000e+00 1.11e+04 1.17e+15  -1.0 2.11e+04    -  1.00e+00 1.96e-03h  9
 176  0.0000000e+00 1.10e+04 1.17e+15  -1.0 2.07e+04    -  1.00e+00 1.96e-03h  9
 177  0.0000000e+00 1.10e+04 1.17e+15  -1.0 2.02e+04    -  1.00e+00 1.96e-03h  9
 178  0.0000000e+00 1.10e+04 1.17e+15  -1.0 1.96e+04    -  1.00e+00 1.96e-03h  9
 179  0.0000000e+00 1.10e+04 1.17e+15  -1.0 1.89e+04    -  1.00e+00 1.96e-03h  9
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 180  0.0000000e+00 1.10e+04 1.17e+15  -1.0 1.80e+04    -  1.00e+00 1.96e-03h  9
 181  0.0000000e+00 1.09e+04 1.17e+15  -1.0 1.72e+04    -  1.00e+00 3.92e-03h  8
 182  0.0000000e+00 1.59e+05 1.38e+15  -1.0 1.67e+04    -  1.00e+00 5.02e-01w  1
 183  0.0000000e+00 9.33e+04 4.59e+15  -1.0 4.75e+04    -  5.08e-01 5.55e-01w  1
 184  0.0000000e+00 9.75e+04 2.25e+18  -1.0 2.67e+03    -  1.02e-01 9.81e-01w  1
 185  0.0000000e+00 1.09e+04 1.17e+15  -1.0 3.80e+06    -  1.00e+00 3.92e-03h  7
 186  0.0000000e+00 1.08e+04

In iteration 287, 2 Slacks too small, adjusting variable bounds
 288  0.0000000e+00 4.58e+03 7.72e+20  -1.0 6.48e+03    -  4.95e-01 1.14e-06H  1
 289  0.0000000e+00 4.58e+03 6.27e+20  -1.0 5.31e+07    -  1.89e-07 9.97e-08f 13
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 290  0.0000000e+00 4.58e+03 4.98e+20  -1.0 4.16e+07    -  2.88e-07 1.27e-07f 13
 291  0.0000000e+00 4.58e+03 3.99e+20  -1.0 4.18e+07    -  4.14e-07 6.33e-08f 14
 292  0.0000000e+00 4.58e+03 3.99e+20  -1.0 3.89e+07    -  4.76e-07 6.81e-08f 14
 293  0.0000000e+00 4.58e+03 3.99e+20  -1.0 3.74e+07    -  5.44e-07 7.07e-08f 14
 294  0.0000000e+00 8.22e+04 9.13e+21  -1.0 3.68e+07    -  6.19e-07 5.89e-04f  1
 295r 0.0000000e+00 8.22e+04 1.00e+03   1.3 0.00e+00  -2.0 0.00e+00 3.59e-07R  6
 296r 0.0000000e+00 2.45e+04 1.10e+03   1.3 2.75e+04    -  4.44e-04 9.93e-04f  1
 297  0.0000000e+00 7.00e+04 3.83e+03  -1.0 9.26e+03    -  9.90e-01 2.78e-01h  1
 298  0.0000000e+00 4.58e+04 1.31e+04  -1.0 1

In [40]:
# For testing purposes
from pyomo.environ import TerminationCondition
assert results.solver.termination_condition == TerminationCondition.optimal

Analyze the results of the square problem
-------------------------

What is the total operating cost? 

In [41]:
print('operating cost = $', value(m.fs.operating_cost))

operating cost = $ 364461.48384096316


For this operating cost, what is the amount of benzene we are able to produce and what purity we are able to achieve? 

In [42]:
m.fs.F102.report()

print()
print('benzene purity = ', value(m.fs.purity))


Unit : fs.F102                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value       : Fixed : Bounds
          Heat Duty :      2263.9 : False : (None, None)
    Pressure Change : -2.0000e+05 :  True : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                               Inlet    Vapor Outlet  Liquid Outlet
    flow_mol_phase_comp ('Liq', 'benzene')     0.10734   1.0000e-08       0.10734  
    flow_mol_phase_comp ('Liq', 'toluene')     0.17017   1.0000e-08       0.17017  
    flow_mol_phase_comp ('Liq', 'hydrogen') 2.7751e-07   1.0000e-08    2.7751e-07  
    flow_mol_phase_comp ('Liq', 'methane')  2.7751e-07   1.0000e-08    2.7751e-07  
    flow_mol_phase_comp ('Vap', 'benzene')  1.0000e-08   4.8301e-08    1.0000e-08  
    flow_mol_phase

Next, let's look at how much benzene we are loosing with the light gases out of F101. IDAES has tools for creating stream tables based on the `Arcs` and/or `Ports` in a flowsheet. Let us create and print a simple stream table showing the stream leaving the reactor and the vapor stream from F101.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
How much benzene are we losing in the F101 vapor outlet stream?
</div>


In [43]:
from idaes.core.util.tables import create_stream_table_dataframe, stream_table_dataframe_to_string

st = create_stream_table_dataframe({"Reactor": m.fs.s05, "Light Gases": m.fs.s06})
st

,Reactor,Light Gases
"flow_mol_phase_comp ('Liq', 'benzene')",0.019349,1.000000e-08
"flow_mol_phase_comp ('Liq', 'toluene')",0.126917,1.000000e-08
"flow_mol_phase_comp ('Liq', 'hydrogen')",0.096911,1.000000e-08
"flow_mol_phase_comp ('Liq', 'methane')",0.404752,1.000000e-08
"flow_mol_phase_comp ('Vap', 'benzene')",0.161092,7.309994e-02
"flow_mol_phase_comp ('Vap', 'toluene')",0.082946,3.969182e-02
"flow_mol_phase_comp ('Vap', 'hydrogen')",0.793635,8.905453e-01
"flow_mol_phase_comp ('Vap', 'methane')",0.305000,7.097520e-01
temperature,767.666119,3.250000e+02
pressure,349999.999191,3.500000e+05


<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
You can querry additional variables here if you like. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>
